In [1]:
import pandas as pd
import csv
import os
import numpy as np
import yfinance as yf
import re

import datetime as dt
import time

import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

pd.options.plotting.backend = 'plotly'

*ADOPCION*

1. EL ORO TIENE CORRELACIÓN POSITIVA CON LA BOLSA
2. EL ORO TIENE CORRELACIÓN NEGATIVA CON LA INFLACIÓN DEL DOLLAR


In [2]:
'''
Precious metals like gold and silver are often seen as a hedge against inflation because they preserve their purchasing power 
for long periods of time. 
So while stocks and other assets may experience large fluctuations, the price of gold may be more stable'''
#Precio energía y crypto?


#Función para round:
rd = lambda x: round(x, 5) if isinstance(x, float) else x

#Lista tickers:
tickers = ['GC=F','CL=F','AAPL','DX-Y.NYB',"BTC-USD",'ETH-USD','DOGE-USD']
nombres = ['Oro','Crudo','Apple','DXY','Bitcoin','Ethereum','DogeCoin']
otros_tickers = [('ADA-USD','Cardano')]

# Fechas a analizar:

start = dt.datetime(2017,11,9)
end = dt.datetime.now()
data= pd.DataFrame()

#Función tickers:

def precios(data,start=start,end=end):

    data = yf.Ticker('^GSPC').history(start=start,end=end).reset_index()[['Date','Close']].rename(columns={'Close':'Spx_500'})
    data['Date'] = data['Date'].dt.date
    data.reset_index(inplace=True,drop=True)
    
    for ticker,nombre in zip(tickers,nombres):
        
        yf_ticker = yf.Ticker(ticker).history(start=start,end=end).reset_index()[['Date','Close']].rename(columns={'Close':nombre})
        yf_ticker['Date'] = yf_ticker['Date'].dt.date
        yf_ticker.reset_index(inplace=True,drop=True)
    
        data = data.merge(yf_ticker,how='right')
        
    return data

data = precios(data)

# Hago forward fill para reemplazar valores del fin de semana en finanza tradicional
data.ffill(inplace=True)
data.bfill(inplace=True)

# Paso a date y redondeo
data.Date = pd.to_datetime(data.Date)
data = data.applymap(rd)

# Quito un outlier en el crudo durante el covid con valor negativo

def outliers_IQR(data,factor):
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)
    iqr = q3-q1

    cutoff = iqr*factor
    lower = q1-cutoff
    upper  = q3+cutoff

    outliers = []
    posicion = []
    
    
    for index,value in enumerate(data):
        if value<lower or value>upper:
            outliers.append(value)
            posicion.append(index)

    return outliers,posicion

outliers_IQR(data.Crudo,2.5)
#data.Crudo.iloc[893] = 17.5

os.makedirs(r'/datos', exist_ok=True)
data.to_csv('datos/stock_price.csv',index=False)
data

,Date,Spx_500,Oro,Crudo,Apple,DXY,Bitcoin,Ethereum,DogeCoin
0,2017-11-09,2584.62012,1285.59998,57.17,41.66130,94.440,7143.58008,320.88400,0.00141
1,2017-11-10,2582.30005,1272.40002,56.74,41.52341,94.390,6618.14014,299.25299,0.00116
2,2017-11-11,2582.30005,1272.40002,56.74,41.52341,94.390,6357.60010,314.68100,0.00120
3,2017-11-12,2582.30005,1272.40002,56.74,41.52341,94.390,5950.06982,307.90799,0.00104
4,2017-11-13,2584.84009,1277.30005,56.76,41.35701,94.490,6559.49023,316.71600,0.00121
...,...,...,...,...,...,...,...,...,...
1851,2022-12-04,4071.69995,1795.90002,79.98,147.81000,104.550,17130.48633,1280.25659,0.10429
1852,2022-12-05,3998.84009,1767.40002,76.93,146.63000,105.290,16974.82617,1259.67676,0.10147
1853,2022-12-06,3941.26001,1769.30005,74.25,142.91000,105.580,17089.50391,1271.65381,0.10028
1854,2022-12-07,3933.91992,1785.50000,72.01,140.94000,105.100,16848.12695,1232.43750,0.09578


In [3]:
outliers_IQR(data.Crudo,2.5)

([-37.63, 123.7, 122.11], [893, 1580, 1672])

In [4]:
#Extraigo una tabla con un listado de países, y el número de usuarios de crypto para visualizar en PowerBI.


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get('https://triple-a.io/crypto-ownership-data/')
time.sleep(6)
html = driver.page_source
html = re.sub(r'<.*?>', lambda g: g.group(0).upper(), html)

df = pd.read_html(html)[0]
count = 8

while count !=0:
    html = driver.page_source
    driver.find_element('xpath','//*[@id="footable_30491"]/tfoot/tr/td/div/ul/li[13]/a').click()
    html = re.sub(r'<.*?>', lambda g: g.group(0).upper(), html)
    df1 = pd.read_html(html)[0]
    df = pd.concat([df,df1],ignore_index=True)
    count-=1
    time.sleep(1)

driver.close()

# Limpio el df y lo paso a csv para visualizar en PowerBI

df.columns = df.columns.droplevel(0)
paises = df.copy()

paises = paises[paises.country != '«‹...12345678...›»'].iloc[:,:-1]
paises.drop_duplicates(inplace=True)

paises['Percentage of the population'] = paises['Percentage of the population'].str.replace('%','').astype('float64')
paises.to_csv('datos/paises.csv')


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 84.0MB/s]
C:\Users\Han\AppData\Local\Temp/ipykernel_31304/3069119962.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [5]:
# Saco DF del top 10 de países por porcentaje. Cambio tipos, añado decimales y %, renombro y reordeno para visualizar en Plotly.

top10 = paises.sort_values('Percentage of the population',ascending=False,ignore_index=True)
top10 = top10.head(10)
top10['Percentage of the population'] = top10['Percentage of the population'].astype(str)+' %'
top10['Number of crypto owners'] = top10['Number of crypto owners'].astype(np.int64)
top10['Number of crypto owners'] = top10['Number of crypto owners'].apply(lambda x: f'{x:,}')


top10.rename(columns={'country':'País','Number of crypto owners':'Personas que poseen criptomonedas',
                      'Percentage of the population':'Porcentaje'},inplace=True)
top10 = top10.iloc[:,[0,2,1]]
top10.insert(0,'Ranking',value=top10.index+1)

top10.to_csv('datos/top10.csv',index=False)

top10

,Ranking,País,Porcentaje,Personas que poseen criptomonedas
0,1,Vietnam,18.73 %,"18,669,973"
1,2,China,16.23 %,"242,564,879"
2,3,Ukraine,14.52 %,"6,018,385"
3,4,United States,13.38 %,"44,816,878"
4,5,South Africa,12.13 %,"7,509,289"
5,6,Singapore,11.05 %,"642,987"
6,7,Kenya,10.71 %,"5,634,955"
7,8,Pakistan,10.62 %,"24,433,805"
8,9,Nigeria,9.55 %,"20,623,813"
9,10,Russia,9.33 %,"13,526,666"


## Análisis de la correlación en Crypto, finanzas tradicionales

In [6]:
#Correlación móvil para ver si BTC se va correlacionando más con la bolsa, y con el dollar.
#Correlación de Bitcoin y distintos assets sobre los 90 últimos días.

btc_sp_corr = data.Bitcoin.rolling(90).corr(data.Spx_500)
btc_oro_corr = data.Bitcoin.rolling(90).corr(data.Oro)
btc_dxy_corr = data.Bitcoin.rolling(90).corr(data.DXY)
spx_oro_corr = data.Spx_500.rolling(90).corr(data.DXY)
oro_dollar_corr = data.Oro.rolling(90).corr(data.DXY)


#Los NA son los X primeros días.
corr_movil = pd.concat([data.Date,btc_sp_corr,btc_oro_corr,btc_dxy_corr,spx_oro_corr],axis=1
                      ).rename(columns={0:'SPX500',1:'Oro',2:'Dollar',3:'SPX_Dollar'}).dropna()

corr_movil.to_csv('datos/corr_movil.csv')


## Volatilidad por meses

In [7]:
#Estudio volatilidad: genero un df de beneficios logarítmicos diarios.

df_log_returns = data.Date

for col in data.iloc[:,1:].columns:
    df_log_returns = pd.concat([df_log_returns,np.log(data[col]/data[col].shift(1))] ,axis=1).dropna()

# Backward fill con valores de 0 (fin de semana en finanza tradicional)
df_log_returns = df_log_returns.replace(0.000000,np.nan).bfill().ffill()

C:\Users\Han\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
# Desviación estándar de los beneficios:
std_diaria = df_log_returns.std()

# Porcentaje de volatilidad anual total:
vol_anual = (std_diaria * np.sqrt(365)).iloc[1:]

display(std_diaria, vol_anual)

Date        535 days 22:11:28.436234048
Spx_500                        0.014161
Oro                            0.009829
Crudo                          0.036194
Apple                          0.021548
DXY                            0.004076
Bitcoin                        0.040526
Ethereum                       0.051645
DogeCoin                       0.079517
dtype: object

Spx_500     0.270554
Oro         0.187788
Crudo       0.691485
Apple       0.411678
DXY         0.077863
Bitcoin     0.774239
Ethereum    0.986667
DogeCoin     1.51917
dtype: object

In [9]:
# Volatilidad por meses, exporto para visualizar en PowerBI:

dias = 60

volatilidad = pd.concat([df_log_returns.Date,(df_log_returns.rolling(window=dias).std()*np.sqrt(dias))],axis=1).dropna()
volatilidad.to_csv('datos/volatilidad_mensual.csv',index=False)

volatilidad

,Date,Spx_500,Oro,Crudo,Apple,DXY,Bitcoin,Ethereum,DogeCoin
60,2018-01-08,0.029515,0.050705,0.097562,0.095020,0.024392,0.546788,0.501455,0.914315
61,2018-01-09,0.029393,0.049824,0.098666,0.094977,0.024516,0.541820,0.503054,0.896164
62,2018-01-10,0.029528,0.049857,0.098906,0.094908,0.024550,0.539220,0.505754,0.898862
63,2018-01-11,0.029974,0.049790,0.098885,0.095026,0.024988,0.547708,0.515055,0.896737
64,2018-01-12,0.030346,0.050482,0.098992,0.095519,0.026576,0.541834,0.520531,0.894404
...,...,...,...,...,...,...,...,...,...
1851,2022-12-04,0.120967,0.081165,0.173403,0.191142,0.061191,0.251238,0.376244,0.628127
1852,2022-12-05,0.121933,0.082641,0.175963,0.191187,0.060632,0.251207,0.376554,0.628493
1853,2022-12-06,0.119332,0.082442,0.169910,0.189282,0.060486,0.250706,0.376462,0.628330
1854,2022-12-07,0.119062,0.080654,0.171226,0.189682,0.060513,0.250955,0.377516,0.630342
